# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.231368522719                   -0.50    6.0
  2   -7.248371517894       -1.77       -1.37    1.0
  3   -7.251264866683       -2.54       -1.69    4.0
  4   -7.251325478436       -4.22       -2.17    2.0
  5   -7.251309600118   +   -4.80       -2.30    3.0
  6   -7.251337994053       -4.55       -3.20    2.0
  7   -7.251338582339       -6.23       -3.46    2.0
  8   -7.251338787298       -6.69       -4.08    1.0
  9   -7.251338797974       -7.97       -4.62    3.0
 10   -7.251338798685       -9.15       -5.32    2.0
 11   -7.251338798701      -10.78       -5.74    3.0
 12   -7.251338798704      -11.56       -6.19    3.0
 13   -7.251338798705      -12.36       -6.86    2.0
 14   -7.251338798705      -14.05       -7.29    4.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06213067222349857
[ Info: Arnoldi iteration step 2: normres = 0.4970482253117415
[ Info: Arnoldi iteration step 3: normres = 0.9353754765404411
[ Info: Arnoldi iteration step 4: normres = 0.322403993923846
[ Info: Arnoldi iteration step 5: normres = 0.4931253638064548
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.64e-02, 7.13e-02, 4.06e-01, 2.65e-01, 1.46e-03)
[ Info: Arnoldi iteration step 6: normres = 0.3183754116886251
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.48e-02, 1.12e-01, 2.46e-01, 9.98e-02, 1.11e-01)
[ Info: Arnoldi iteration step 7: normres = 0.07743834206109726
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (5.55e-04, 9.08e-03, 1.61e-02, 3.88e-02, 5.79e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11468282245112647
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.79e-05